Дорохов Георгий 3530904/90106

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
# Read, then decode for py2 compat.
text = open("chehov.txt", 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 103008 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

В больничном дворе стоит небольшой флигель, окруженный целым лесом репейника, крапивы и дикой конопли. Крыша на нем ржавая, труба наполовину обвалилась, ступеньки у крыльца сгнили и поросли травой, а от штукатурки остались одни только следы. Передним


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

93 unique characters


In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[0, 0, 0, 0, 22, 0, 0], [0, 0, 0]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'e', b'[UNK]', b'[UNK]'],
 [b'[UNK]', b'[UNK]', b'[UNK]']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'[UNK][UNK][UNK][UNK]e[UNK][UNK]', b'[UNK][UNK][UNK]'],
      dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(103008,), dtype=int64, numpy=array([34,  2, 59, ..., 68, 58,  8])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

В
 
б
о
л
ь
н
и
ч
н


In [15]:
seq_length = 100


In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xd0\x92' b' ' b'\xd0\xb1' b'\xd0\xbe' b'\xd0\xbb' b'\xd1\x8c'
 b'\xd0\xbd' b'\xd0\xb8' b'\xd1\x87' b'\xd0\xbd' b'\xd0\xbe' b'\xd0\xbc'
 b' ' b'\xd0\xb4' b'\xd0\xb2' b'\xd0\xbe' b'\xd1\x80' b'\xd0\xb5' b' '
 b'\xd1\x81' b'\xd1\x82' b'\xd0\xbe' b'\xd0\xb8' b'\xd1\x82' b' '
 b'\xd0\xbd' b'\xd0\xb5' b'\xd0\xb1' b'\xd0\xbe' b'\xd0\xbb' b'\xd1\x8c'
 b'\xd1\x88' b'\xd0\xbe' b'\xd0\xb9' b' ' b'\xd1\x84' b'\xd0\xbb'
 b'\xd0\xb8' b'\xd0\xb3' b'\xd0\xb5' b'\xd0\xbb' b'\xd1\x8c' b',' b' '
 b'\xd0\xbe' b'\xd0\xba' b'\xd1\x80' b'\xd1\x83' b'\xd0\xb6' b'\xd0\xb5'
 b'\xd0\xbd' b'\xd0\xbd' b'\xd1\x8b' b'\xd0\xb9' b' ' b'\xd1\x86'
 b'\xd0\xb5' b'\xd0\xbb' b'\xd1\x8b' b'\xd0\xbc' b' ' b'\xd0\xbb'
 b'\xd0\xb5' b'\xd1\x81' b'\xd0\xbe' b'\xd0\xbc' b' ' b'\xd1\x80'
 b'\xd0\xb5' b'\xd0\xbf' b'\xd0\xb5' b'\xd0\xb9' b'\xd0\xbd' b'\xd0\xb8'
 b'\xd0\xba' b'\xd0\xb0' b',' b' ' b'\xd0\xba' b'\xd1\x80' b'\xd0\xb0'
 b'\xd0\xbf' b'\xd0\xb8' b'\xd0\xb2' b'\xd1\x8b' b' ' b'\xd0\xb8' b' '
 b'\xd0\xb4' b'\x

In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\xd0\x92 \xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xb8\xd1\x87\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xb2\xd0\xbe\xd1\x80\xd0\xb5 \xd1\x81\xd1\x82\xd0\xbe\xd0\xb8\xd1\x82 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd1\x88\xd0\xbe\xd0\xb9 \xd1\x84\xd0\xbb\xd0\xb8\xd0\xb3\xd0\xb5\xd0\xbb\xd1\x8c, \xd0\xbe\xd0\xba\xd1\x80\xd1\x83\xd0\xb6\xd0\xb5\xd0\xbd\xd0\xbd\xd1\x8b\xd0\xb9 \xd1\x86\xd0\xb5\xd0\xbb\xd1\x8b\xd0\xbc \xd0\xbb\xd0\xb5\xd1\x81\xd0\xbe\xd0\xbc \xd1\x80\xd0\xb5\xd0\xbf\xd0\xb5\xd0\xb9\xd0\xbd\xd0\xb8\xd0\xba\xd0\xb0, \xd0\xba\xd1\x80\xd0\xb0\xd0\xbf\xd0\xb8\xd0\xb2\xd1\x8b \xd0\xb8 \xd0\xb4\xd0\xb8\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xba\xd0\xbe\xd0\xbd\xd0\xbe\xd0\xbf\xd0\xbb\xd0\xb8'
b'. \xd0\x9a\xd1\x80\xd1\x8b\xd1\x88\xd0\xb0 \xd0\xbd\xd0\xb0 \xd0\xbd\xd0\xb5\xd0\xbc \xd1\x80\xd0\xb6\xd0\xb0\xd0\xb2\xd0\xb0\xd1\x8f, \xd1\x82\xd1\x80\xd1\x83\xd0\xb1\xd0\xb0 \xd0\xbd\xd0\xb0\xd0\xbf\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb8\xd0\xbd\xd1\x83 \xd0\xbe\xd0\xb1\xd0\xb2\xd0\xb0\x

In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xd0\x92 \xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xb8\xd1\x87\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xb2\xd0\xbe\xd1\x80\xd0\xb5 \xd1\x81\xd1\x82\xd0\xbe\xd0\xb8\xd1\x82 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd1\x88\xd0\xbe\xd0\xb9 \xd1\x84\xd0\xbb\xd0\xb8\xd0\xb3\xd0\xb5\xd0\xbb\xd1\x8c, \xd0\xbe\xd0\xba\xd1\x80\xd1\x83\xd0\xb6\xd0\xb5\xd0\xbd\xd0\xbd\xd1\x8b\xd0\xb9 \xd1\x86\xd0\xb5\xd0\xbb\xd1\x8b\xd0\xbc \xd0\xbb\xd0\xb5\xd1\x81\xd0\xbe\xd0\xbc \xd1\x80\xd0\xb5\xd0\xbf\xd0\xb5\xd0\xb9\xd0\xbd\xd0\xb8\xd0\xba\xd0\xb0, \xd0\xba\xd1\x80\xd0\xb0\xd0\xbf\xd0\xb8\xd0\xb2\xd1\x8b \xd0\xb8 \xd0\xb4\xd0\xb8\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xba\xd0\xbe\xd0\xbd\xd0\xbe\xd0\xbf\xd0\xbb'
Target: b' \xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xb8\xd1\x87\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xb2\xd0\xbe\xd1\x80\xd0\xb5 \xd1\x81\xd1\x82\xd0\xbe\xd0\xb8\xd1\x82 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd1\x88\xd0\xbe\xd0\xb9 \xd1\x84\xd0\xbb\xd0\xb8\xd0\xb3\xd0\xb5\xd0\x

2022-11-21 20:43:25.216929: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 94) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24064     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  96350     
                                                                 
Total params: 4,058,718
Trainable params: 4,058,718
Non-trainable params: 0
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [29]:
sampled_indices

array([ 7, 25, 46,  2, 50,  3, 58, 77, 82, 57, 54, 14, 37, 35, 93, 13, 11,
       57, 50, 32, 31, 88, 33, 74,  1, 33, 17, 69, 64, 54, 73,  5, 29, 34,
       36,  3, 12, 41, 22, 34, 24, 91, 55, 16, 73, 16,  7, 54, 74, 78, 14,
       70, 86, 36,  7, 86, 16, 30, 81, 48, 39, 22, 86, 11, 54,  2, 68,  4,
       67, 60, 80, 48,  1, 49,  0, 31, 58, 14, 33, 28, 37, 33, 77, 55, 93,
       80, 82, 90, 50, 88, 80, 24, 26, 58, 74, 29, 55, 49, 47, 19])

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' \xd0\xad\xd1\x82\xd0\xbe\xd0\xb9 \xd0\xb5\xd1\x80\xd1\x83\xd0\xbd\xd0\xb4\xd1\x8b \xd1\x8f \xd0\xbd\xd0\xb5 \xd0\xbf\xd0\xbe\xd0\xbd\xd0\xb8\xd0\xbc\xd0\xb0\xd1\x8e\xe2\x80\xa6 \xe2\x80\x93 \xd0\xb3\xd0\xbb\xd1\x83\xd0\xbf\xd0\xbe \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb3\xd0\xbe\xd0\xb2\xd0\xbe\xd1\x80\xd0\xb8\xd0\xbb \xd0\x98\xd0\xb2\xd0\xb0\xd0\xbd \xd0\x94\xd0\xbc\xd0\xb8\xd1\x82\xd1\x80\xd0\xb8\xd1\x87 \xd0\xb8 \xd1\x81\xd0\xb5\xd0\xbb \xd0\xbd\xd0\xb0 \xd1\x81\xd0\xb2\xd0\xbe\xd1\x8e \xd0\xba\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c.\n\n\xd0\x9c\xd0\xbe\xd0\xb9\xd1\x81\xd0\xb5\xd0\xb9\xd0\xba\xd0\xb0, \xd0\xba\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb3'

Next Char Predictions:
 b'-n\xd0\x9f \xd0\xa3!\xd0\xb0\xd1\x83\xd1\x88\xd0\xaf\xd0\xa7:\xd0\x95\xd0\x93\xe2\x84\x9683\xd0\xaf\xd0\xa3\xd0\x90\xc2\xbb\xd1\x8e\xd0\x91\xd1\x80\n\xd0\x91I\xd0\xbb\xd0\xb6\xd0\xa7\xd0\xbf)u\xd0\x92\xd0\x94!6\xd0\x9ae\xd0\x92m\xe2\x80\x93\xd0\xa8?\xd0\xbf?-\xd0\xa7\xd1\x80\xd1\x84:\xd0\xb

In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 94)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5441227, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

94.07786

In [34]:
model.compile(optimizer='adam', loss=loss)

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [36]:
EPOCHS = 80

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/80
15/15 [==============================] - 20s 1s/step - loss: 4.3811
Epoch 2/80
15/15 [==============================] - 23s 2s/step - loss: 3.4321
Epoch 3/80
15/15 [==============================] - 22s 1s/step - loss: 3.0973
Epoch 4/80
15/15 [==============================] - 21s 1s/step - loss: 2.8716
Epoch 5/80
15/15 [==============================] - 22s 1s/step - loss: 2.6923
Epoch 6/80
15/15 [==============================] - 21s 1s/step - loss: 2.5846
Epoch 7/80
15/15 [==============================] - 20s 1s/step - loss: 2.5175
Epoch 8/80
15/15 [==============================] - 18s 1s/step - loss: 2.4691
Epoch 9/80
15/15 [==============================] - 18s 1s/step - loss: 2.4272
Epoch 10/80
15/15 [==============================] - 17s 1s/step - loss: 2.3872
Epoch 11/80
15/15 [==============================] - 17s 1s/step - loss: 2.3477
Epoch 12/80
15/15 [==============================] - 18s 1s/step - loss: 2.3091
Epoch 13/80
15/15 [==============================

In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [40]:
start = time.time()
states = None
next_char = tf.constant(['Текст: '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Текст: – сказал он дихно: образаяные к нему заказное случай. Сонов же в конце концов! входи с Андрею Ефимычу. – Здравствуйте, мой дорогой! Небось я уже надоел в сливении, но послушантноет он и Моханился!

А вде голожели из теллого подимался а пехочаться. Поталий вам, но свое одному не завозбу. О день мниго поди подозритель, что он бриносли обеданец? Нектор и считается больным и не выходил из номера. Он лежал лицом к спинке дивать и рассергелоный и сказал:

– Как бы земле вообще тюмься, потому станаям и скоро построен и постучил объятьте и сказал:

– Честь спасена! Едемте, мой друг! Т эазотом, приходить к нему, возможнаясь, что в ней славо, что тыкая у вас бредства теликать с места, а Дмущети стартие еежу спиле, с теремной ворхуди разводко и тю еть тесло и похожа на жизнь аме? Вод всегда подозреть, с тоже име поволол хорошот рубахилься за рошли из толкость с достави. К нему еще раз в жизни ему стало горько жаль медицину.

– Но почему-то это положительны, потчим на железно остолительный 

In [41]:
start = time.time()
states = None
next_char = tf.constant([':', ':', ':', ':', ':'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b': \xc2\xab\xd0\x9a\xd0\xb0\xd0\xba \xd0\xbe\xd0\xbd \xd0\xbd\xd0\xb5 \xd0\xb2 \xd0\xbe\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xb4\xd0\xbe\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd1\x81\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x8f \xd0\xb8 \xd0\xba\xd0\xb0\xd0\xba-\xd1\x82\xd0\xbe \xd0\xb3\xd1\x80\xd0\xbe\xd0\xbc\xd0\xba\xd0\xbe \xd1\x85\xd0\xb0\xd1\x80\xd0\xb0\xd0\xba\xd1\x82\xd0\xb5-\xd0\xbc\xd0\xbe \xd0\xb1\xd1\x8b\xd0\xbb\xd0\xbe \xd0\xbf\xd0\xbe \xd0\xba\xd0\xbe\xd1\x82\xd0\xbf\xd1\x80\xd0\xb0 \xd0\xb8 \xd1\x87\xd1\x82\xd0\xbe \xd0\xb5\xd0\xb5 \xd0\xbd\xd0\xb0\xd0\xb4\xd0\xbe \xd0\xb1\xd1\x8b\xd0\xbb \xd1\x83\xd0\xb6\xd0\xb5 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xb5\xd1\x8e\xd1\x81\xd1\x8c \xd0\xba\xd1\x83\xd1\x87\xd0\xb8-\xd1\x82\xd0\xbe\xd0\xb8 \xd0\xb8\xd0\xbb\xd0\xb8 \xd0\xba\xd0\xb0\xd0\xba \xd0\xb1\xd1\x8b\xd1\x81\xd1\x82\xd1\x80\xd0\xbe \xd0\xb8 \xd0\xb2 \xd0\xba\xd0\xb8\xd1\x85\xd0\xbe\xd0\xb5 \xd0\xbb\xd0\xb5\xd0\xb6\xd0\xb8. \xd0\x9d\xd1\x83\xd0\xb1\xd1\x80\xd0\xbe\xd0\xb6

In [42]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [43]:
states = None
next_char = tf.constant(['Текст:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Текст: «Боясь он полож, знееста, как в тюрьме вам бы чувство у него было одно: досада на Михаила Аверьяныч


In [44]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [45]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [46]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [47]:
model.fit(dataset, epochs=1)

15/15 [==============================] - 18s 1s/step - loss: 4.4159


In [48]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.0691

Epoch 1 Loss: 3.5263
Time taken for 1 epoch 17.78 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 3.2185

Epoch 2 Loss: 3.1054
Time taken for 1 epoch 16.76 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.9515

Epoch 3 Loss: 2.8641
Time taken for 1 epoch 16.43 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 2.7593

Epoch 4 Loss: 2.6907
Time taken for 1 epoch 16.65 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 2.6448

Epoch 5 Loss: 2.5854
Time taken for 1 epoch 17.19 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 2.5607

Epoch 6 Loss: 2.5180
Time taken for 1 epoch 17.74 sec
________________________________________________________________________________
Epoch 7 Batch 0 